# Lesson29

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

from torch.utils.tensorboard import SummaryWriter

C:\Users\user\.conda\envs\torch\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\.conda\envs\torch\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\.conda\envs\torch\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\.conda\envs\torch\lib\site-packages\tensorboard\co

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.5.0
0.6.0


In [3]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST'
    ,train = True
    ,download = True
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [5]:
class Network(nn.Module):#line 1
    def __init__(self):
        super(Network,self).__init__()# line 3
        #第一层的输入为1，参数依赖数据集（用的是单色图像）
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        
        
        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60) 
        self.out = nn.Linear(in_features=60,out_features=10)
        #最后一层输出为10，分类有10个类别，也依赖数据集
    def forward(self,t):  
        #(1) input layer
        t = t 
        
        #(2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
         #(3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        #(4)hidden liner layer
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        #(5)hidden liner layer
        t = self.fc2(t)
        t = F.relu(t)
        
        #(6) outpur layer
        t = self.out(t)
        #t = F.softmax(t,dim=1)
        
        return t

In [6]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)

## Starting out with Tensorboard(Network Graph and Images)

In [7]:
tb = SummaryWriter()

network = Network()
images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images',grid)
tb.add_graph(network,images)
tb.close()

In [13]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)
optimizer = optim.Adam(network.parameters(),lr=0.01)

images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images',grid)
tb.add_graph(network,images)

for epoch in range(10):

    total_loss=0
    total_correct=0

    for batch in train_loader:#get batch
        images,labels = batch

        preds = network(images)#pass batch
        loss = F.cross_entropy(preds,labels)#calculate loss

        optimizer.zero_grad()
        loss.backward()#calculate gradients
        optimizer.step()#update weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
    
    tb.add_scalar('Loss',total_loss,epoch)
    tb.add_scalar('Number Correct',total_correct,epoch)
    tb.add_scalar('Accuracy',total_correct / len(train_set),epoch)
    
    #tb.add_histogram('conv1.bias',network.conv1.bias,epoch)
    #tb.add_histogram('conv1.weight',network.conv1.weight,epoch)
    #tb.add_histogram('conv1.weight.grad',network.conv1.weight.grad,epoch)
    
    for name,weight in network.named_parameters():
        tb.add_histogram(name,weight,epoch)
        tb.add_histogram(f'{name}.grad',weight.grad,epoch) 

    print("epoch:",epoch,"total_correct:",total_correct,"loss:",total_loss)
    
tb.close()

epoch: 0 total_correct: 46630 loss: 351.0307514369488
epoch: 1 total_correct: 51243 loss: 237.669718131423
epoch: 2 total_correct: 51797 loss: 221.65378358960152
epoch: 3 total_correct: 52117 loss: 213.49102199077606
epoch: 4 total_correct: 52459 loss: 205.06129117310047
epoch: 5 total_correct: 52434 loss: 204.5997589379549
epoch: 6 total_correct: 52750 loss: 196.88377752900124
epoch: 7 total_correct: 52767 loss: 195.12314468622208
epoch: 8 total_correct: 52977 loss: 191.36762250959873
epoch: 9 total_correct: 52926 loss: 189.84433709084988


# Lesson30 

In [14]:
batch_size_list = [100,1000,10000]
lr_list = [.01,.001,.0001,.00001]

In [ ]:
for batch_size in batch_size_list:
    for lr in lr_list:

        network = Network()
        train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)
        optimizer = optim.Adam(network.parameters(),lr=lr)

        images,labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment = f'batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images',grid)
        tb.add_graph(network,images)

        for epoch in range(10):

            total_loss=0
            total_correct=0

            for batch in train_loader:#get batch
                images,labels = batch

                preds = network(images)#pass batch
                loss = F.cross_entropy(preds,labels)#calculate loss

                optimizer.zero_grad()
                loss.backward()#calculate gradients
                optimizer.step()#update weights

                total_loss += loss.item()*batch_size
                total_correct += get_num_correct(preds,labels)
    
            tb.add_scalar('Loss',total_loss,epoch)
            tb.add_scalar('Number Correct',total_correct,epoch)
            tb.add_scalar('Accuracy',total_correct / len(train_set),epoch)
    
    #tb.add_histogram('conv1.bias',network.conv1.bias,epoch)
    #tb.add_histogram('conv1.weight',network.conv1.weight,epoch)
    #tb.add_histogram('conv1.weight.grad',network.conv1.weight.grad,epoch)
    
            for name,weight in network.named_parameters():
                tb.add_histogram(name,weight,epoch)
                tb.add_histogram(f'{name}.grad',weight.grad,epoch) 

            print("epoch:",epoch,"total_correct:",total_correct,"loss:",total_loss)
    
        tb.close()

In [15]:
from itertools import product

In [16]:
parameters = dict(
    lr=[.01,.001]
    ,batch_size=[10,100,1000]
    ,shuffle=[True,False]
)

In [17]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [10, 100, 1000], [True, False]]

In [18]:
for lr,batch_size,shuffle in product(*param_values):
    print(lr,batch_size,shuffle)

0.01 10 True
0.01 10 False
0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.001 10 True
0.001 10 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False


In [21]:
for lr,batch_size,shuffle in product(*param_values):

    comment = f'batch_size={batch_size} lr={lr} shuffle={shuffle}'
    network = Network()
    train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=shuffle)
    optimizer = optim.Adam(network.parameters(),lr=lr)

    images,labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)

    #comment = f'batch_size={batch_size} lr={lr}'
    tb = SummaryWriter(comment=comment)
    tb.add_image('images',grid)
    tb.add_graph(network,images)

    for epoch in range(2):

        total_loss=0
        total_correct=0

        for batch in train_loader:#get batch
            images,labels = batch

            preds = network(images)#pass batch
            loss = F.cross_entropy(preds,labels)#calculate loss

            optimizer.zero_grad()
            loss.backward()#calculate gradients
            optimizer.step()#update weights

            total_loss += loss.item()*batch_size
            total_correct += get_num_correct(preds,labels)
    
        tb.add_scalar('Loss',total_loss,epoch)
        tb.add_scalar('Number Correct',total_correct,epoch)
        tb.add_scalar('Accuracy',total_correct / len(train_set),epoch)
    
    #tb.add_histogram('conv1.bias',network.conv1.bias,epoch)
    #tb.add_histogram('conv1.weight',network.conv1.weight,epoch)
    #tb.add_histogram('conv1.weight.grad',network.conv1.weight.grad,epoch)
    
        for name,weight in network.named_parameters():
            tb.add_histogram(name,weight,epoch)
            tb.add_histogram(f'{name}.grad',weight.grad,epoch) 

        print("epoch:",epoch,"total_correct:",total_correct,"loss:",total_loss)
    
    tb.close()

epoch: 0 total_correct: 45060 loss: 39601.12988737412
epoch: 1 total_correct: 48339 loss: 32289.857091507874
epoch: 0 total_correct: 45814 loss: 37922.699721083045
epoch: 1 total_correct: 48568 loss: 31723.53576716734
epoch: 0 total_correct: 47386 loss: 33439.22770768404
epoch: 1 total_correct: 51557 loss: 22848.083353042603
epoch: 0 total_correct: 45985 loss: 36910.90911030769
epoch: 1 total_correct: 51182 loss: 24044.046126306057
epoch: 0 total_correct: 38412 loss: 57546.21088504791
epoch: 1 total_correct: 47899 loss: 31707.999616861343
epoch: 0 total_correct: 35092 loss: 65412.20438480377
epoch: 1 total_correct: 45817 loss: 36716.35419130325
epoch: 0 total_correct: 47462 loss: 33414.3913878873
epoch: 1 total_correct: 51893 loss: 21822.1542068664
epoch: 0 total_correct: 46979 loss: 34627.42826823611
epoch: 1 total_correct: 51630 loss: 22692.014530757442
epoch: 0 total_correct: 42561 loss: 46678.36547791958
epoch: 1 total_correct: 48328 loss: 31308.32652449608
epoch: 0 total_correct: 